## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Almeirim,BR,2022-08-08 16:10:26,-1.5233,-52.5817,90.91,47,7,6.60,clear sky
1,1,Arraial Do Cabo,BR,2022-08-08 16:15:09,-22.9661,-42.0278,75.16,78,75,10.36,light rain
2,2,Bluff,NZ,2022-08-08 16:15:09,-46.6000,168.3333,34.81,67,2,18.01,clear sky
3,3,Ushuaia,AR,2022-08-08 16:11:37,-54.8000,-68.3000,40.66,52,20,3.44,few clouds
4,4,Carman,CA,2022-08-08 16:15:10,49.4992,-98.0016,75.54,60,30,6.91,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
choose_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
choose_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Arraial Do Cabo,BR,2022-08-08 16:15:09,-22.9661,-42.0278,75.16,78,75,10.36,light rain
4,4,Carman,CA,2022-08-08 16:15:10,49.4992,-98.0016,75.54,60,30,6.91,scattered clouds
5,5,Kapaa,US,2022-08-08 16:12:19,22.0752,-159.3190,75.52,84,40,13.80,scattered clouds
10,10,Cabo San Lucas,MX,2022-08-08 16:12:28,22.8909,-109.9124,86.61,74,100,10.36,overcast clouds
11,11,Chodavaram,IN,2022-08-08 16:15:12,17.8333,82.9500,78.64,93,100,8.46,overcast clouds
12,12,Kavaratti,IN,2022-08-08 16:15:12,10.5669,72.6420,80.71,83,100,20.62,overcast clouds
13,13,Jamestown,US,2022-08-08 16:13:07,42.0970,-79.2353,77.02,72,100,12.66,light rain
16,16,Georgetown,MY,2022-08-08 16:11:38,5.4112,100.3354,84.88,69,20,2.30,few clouds
17,17,Jiaojiang,CN,2022-08-08 16:15:13,28.6803,121.4428,79.61,90,10,7.94,clear sky
20,20,Hithadhoo,MV,2022-08-08 16:15:14,-0.6000,73.0833,82.18,65,100,2.77,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
choose_cities_df.count()

City_ID                256
City                   256
Country                253
Date                   256
Lat                    256
Lng                    256
Max Temp               256
Humidity               256
Cloudiness             256
Wind Speed             256
Current Description    256
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
vacation_cities_df=choose_cities_df.dropna()

In [7]:
vacation_cities_df.count()

City_ID                253
City                   253
Country                253
Date                   253
Lat                    253
Lng                    253
Max Temp               253
Humidity               253
Cloudiness             253
Wind Speed             253
Current Description    253
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = vacation_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Arraial Do Cabo,BR,75.16,light rain,-22.9661,-42.0278,
4,Carman,CA,75.54,scattered clouds,49.4992,-98.0016,
5,Kapaa,US,75.52,scattered clouds,22.0752,-159.3190,
10,Cabo San Lucas,MX,86.61,overcast clouds,22.8909,-109.9124,
11,Chodavaram,IN,78.64,overcast clouds,17.8333,82.9500,
12,Kavaratti,IN,80.71,overcast clouds,10.5669,72.6420,
13,Jamestown,US,77.02,light rain,42.0970,-79.2353,
16,Georgetown,MY,84.88,few clouds,5.4112,100.3354,
17,Jiaojiang,CN,79.61,clear sky,28.6803,121.4428,
20,Hithadhoo,MV,82.18,overcast clouds,-0.6000,73.0833,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found....moving on....")

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")  
        

Hotel not found....moving on....
Hotel not found....moving on....
Hotel not found....moving on....
Hotel not found....moving on....
Hotel not found....moving on....
Hotel not found....moving on....
Hotel not found....moving on....
Hotel not found....moving on....
Hotel not found....moving on....
Hotel not found....moving on....
Hotel not found....moving on....
Hotel not found....moving on....
Hotel not found....moving on....
Hotel not found....moving on....
Hotel not found....moving on....
Hotel not found....moving on....
Hotel not found....moving on....
Hotel not found....moving on....
-----------------------------
Data Retrieval Complete      
-----------------------------


In [16]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Arraial Do Cabo,BR,75.16,light rain,-22.9661,-42.0278,Pousada Porto Praia
4,Carman,CA,75.54,scattered clouds,49.4992,-98.0016,"Bell Aura Bed, Breakfast & Bistro"
5,Kapaa,US,75.52,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort Hotel
10,Cabo San Lucas,MX,86.61,overcast clouds,22.8909,-109.9124,Hotel Tesoro Los Cabos
11,Chodavaram,IN,78.64,overcast clouds,17.8333,82.9500,Ganesh Defence Academy chodavadaram hostel


In [17]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()

In [18]:
#7a. Checking the data in new Data Frame
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Arraial Do Cabo,BR,75.16,light rain,-22.9661,-42.0278,Pousada Porto Praia
4,Carman,CA,75.54,scattered clouds,49.4992,-98.0016,"Bell Aura Bed, Breakfast & Bistro"
5,Kapaa,US,75.52,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort Hotel
10,Cabo San Lucas,MX,86.61,overcast clouds,22.8909,-109.9124,Hotel Tesoro Los Cabos
11,Chodavaram,IN,78.64,overcast clouds,17.8333,82.9500,Ganesh Defence Academy chodavadaram hostel


In [19]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt>{Current Description} & {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [21]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))